In [ ]:
import pandas as pd
import seaborn as sns

from keplergl import KeplerGl
import base64
import IPython


# Load the data

In [ ]:
train_2016 = pd.read_csv('https://drive.google.com/uc?id=15GlcdLJ79bc5_WhVNViepQaXvsE1vYb8')
# train_2017 = pd.read_csv('https://drive.google.com/uc?id=1plmVfkf9wn5zXxjsJFKBkOrNmF-E31CT')


In [ ]:
properties_2016 = pd.read_csv('/Users/yang/Downloads/properties_2016.csv')
# properties_2017 = pd.read_csv('/Users/yang/Downloads/properties_2017.csv')


# Sanity check data

In [ ]:
# check head of the data to make sure we're importing what we're expecting
properties_2016.head()

In [ ]:
train_2016.head()

In [ ]:
# let's check what percentage of the data is null
coverage_stats = (properties_2016.isnull().sum() / properties_2016.shape[0])
coverage_stats.sort_values(ascending=False)

In [ ]:
# we have about 24 okay features, then coverage really starts to tank
coverage_stats.sort_values(ascending=False).plot.bar(figsize=(30,10))

# Join training data to features

In [ ]:
training_data = pd.merge(train_2016, properties_2016, on=['parcelid'], how='inner')

In [ ]:
coverage_stats = (training_data.isnull().sum() / training_data.shape[0])
training_data.sort_values(ascending=False)

In [ ]:
coverage_stats.sort_values(ascending=False).plot.bar(figsize=(30,10))

In [ ]:
# let's check where these homes actually are
map_data = training_data.assign(latitude=training_data.latitude/1e6, longitude=training_data.longitude/1e6)[['parcelid', 'latitude', 'longitude']].sample(10000)

properties_map = KeplerGl(
    data={
        'properties': map_data
    },
    config={
        'version': 'v1',
        'config': {
            'mapState': {
                'latitude': map_data.latitude.mean(),
                'longitude': map_data.longitude.mean()
            }
        }
    }
)
b64d_html = base64.b64encode(str(properties_map._repr_html_(),'utf-8').encode('utf-8')).decode('utf-8')
framed_html = f'<iframe src="data:text/html;base64,{b64d_html}" style="width:95%; height: 600px">'
IPython.display.HTML(framed_html)

